<a href="https://colab.research.google.com/github/sahilkholkar/PROJECT/blob/main/Anomaly_Detection_in_Surveillance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Define which actions are normal and which are anomalous
NORMAL_CLASSES = ['walk', 'stand', 'sit', 'run']
ANOMALY_CLASSES = ['struggle', 'throw', 'sneak', 'lying_down', 'hit', 'kick', 'gun', 'grab', 'fall']
VIDEO_DIR = '/content/drive/MyDrive/Anomaly Detection in Surveillance/Videos'


In [3]:
import os
import cv2

def extract_frames(video_path, output_dir, label, max_frames=100):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (128, 128))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        filename = os.path.join(output_dir, f"{label}_{count}.jpg")
        cv2.imwrite(filename, frame)
        count += 1
    cap.release()

# Loop through all video folders and extract frames
for folder in os.listdir(VIDEO_DIR):
    full_folder_path = os.path.join(VIDEO_DIR, folder)
    if os.path.isdir(full_folder_path):
        label = folder.lower()
        for file in os.listdir(full_folder_path):
            if file.endswith('.mp4') or file.endswith('.avi'):
                video_path = os.path.join(full_folder_path, file)
                if label in NORMAL_CLASSES:
                    extract_frames(video_path, 'data/train/normal', label)
                    extract_frames(video_path, 'data/test/normal', label)
                elif label in ANOMALY_CLASSES:
                    extract_frames(video_path, 'data/test/anomaly', label)


In [4]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class FrameDataset(Dataset):
    def __init__(self, folder):
        self.folder = folder
        self.files = [os.path.join(folder, f) for f in os.listdir(folder)]
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = Image.open(self.files[idx])
        return self.transform(img)
